In [2]:
!pip install numpy
!pip install matplotlib
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=e25329b63d136655f043be05146980ccb3355df136bfac7aefa64e921a3402e2
  Stored in directory: c:\users\arttu\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer as Imputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
df = pd.read_csv("credit_train.csv")

In [5]:
df = df[:10000]

In [6]:
df.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6.0,1.0,228190.0,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9.0,0.0,256329.0,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15.0,0.0,253460.0,427174.0,0.0,0.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan ID                       10000 non-null  object 
 1   Customer ID                   10000 non-null  object 
 2   Loan Status                   10000 non-null  object 
 3   Current Loan Amount           10000 non-null  float64
 4   Term                          10000 non-null  object 
 5   Credit Score                  8063 non-null   float64
 6   Annual Income                 8063 non-null   float64
 7   Years in current job          9570 non-null   object 
 8   Home Ownership                10000 non-null  object 
 9   Purpose                       10000 non-null  object 
 10  Monthly Debt                  10000 non-null  float64
 11  Years of Credit History       10000 non-null  float64
 12  Months since last delinquent  4702 non-null   float64
 13  Nu

In [8]:
cols_to_remove = ['Loan ID', 'Customer ID']
data = df.drop(cols_to_remove, axis=1)

In [9]:
#Imputation strategy: Replace Non-existing values with the respective column's average
cols_to_clean =['Current Loan Amount','Credit Score','Annual Income','Years of Credit History',
        'Months since last delinquent','Number of Open Accounts','Number of Credit Problems',
       'Current Credit Balance','Maximum Open Credit','Bankruptcies','Tax Liens']

imputer = Imputer()
data[cols_to_clean] = imputer.fit_transform(data[cols_to_clean])
data[cols_to_clean] = data[cols_to_clean].astype(int)

#Remove rows that still contain at least one NaN value
data=data.dropna()

In [10]:
data

,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,Fully Paid,445412,Short Term,709,1167493,8 years,Home Mortgage,Home Improvements,5214.74,17,35,6,1,228190,416746,1,0
1,Fully Paid,262328,Short Term,1105,1373190,10+ years,Home Mortgage,Debt Consolidation,33295.98,21,8,35,0,229976,850784,0,0
2,Fully Paid,99999999,Short Term,741,2231892,8 years,Own Home,Debt Consolidation,29200.53,14,29,18,1,297996,750090,0,0
3,Fully Paid,347666,Long Term,721,806949,3 years,Own Home,Debt Consolidation,8741.90,12,35,9,0,256329,386958,0,0
4,Fully Paid,176220,Short Term,1105,1373190,5 years,Rent,Debt Consolidation,20639.70,6,35,15,0,253460,427174,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Charged Off,143594,Short Term,736,1024917,4 years,Home Mortgage,other,12982.13,8,40,14,0,235334,604186,0,0
9996,Fully Paid,400158,Long Term,1105,1373190,< 1 year,Rent,Debt Consolidation,7084.53,15,35,3,0,332557,422202,0,0
9997,Fully Paid,154462,Short Term,751,833796,9 years,Home Mortgage,other,10630.88,17,35,7,0,76494,517902,0,0
9998,Fully Paid,264484,Short Term,733,1427565,2 years,Rent,Debt Consolidation,28313.42,10,35,22,0,324862,1245530,0,0


In [11]:

#Convert our target attribute to numerical values
y = []
for i in data['Loan Status']:
    if i == 'Fully Paid':
        y.append(1)
    else:
        y.append(0)

data = data.drop('Loan Status', axis=1)

In [12]:
data.head()

,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,445412,Short Term,709,1167493,8 years,Home Mortgage,Home Improvements,5214.74,17,35,6,1,228190,416746,1,0
1,262328,Short Term,1105,1373190,10+ years,Home Mortgage,Debt Consolidation,33295.98,21,8,35,0,229976,850784,0,0
2,99999999,Short Term,741,2231892,8 years,Own Home,Debt Consolidation,29200.53,14,29,18,1,297996,750090,0,0
3,347666,Long Term,721,806949,3 years,Own Home,Debt Consolidation,8741.90,12,35,9,0,256329,386958,0,0
4,176220,Short Term,1105,1373190,5 years,Rent,Debt Consolidation,20639.70,6,35,15,0,253460,427174,0,0


In [13]:
# Convert categorical attributes to numerical values

print(data.info())
data = pd.get_dummies(data)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9570 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Current Loan Amount           9570 non-null   int32  
 1   Term                          9570 non-null   object 
 2   Credit Score                  9570 non-null   int32  
 3   Annual Income                 9570 non-null   int32  
 4   Years in current job          9570 non-null   object 
 5   Home Ownership                9570 non-null   object 
 6   Purpose                       9570 non-null   object 
 7   Monthly Debt                  9570 non-null   float64
 8   Years of Credit History       9570 non-null   int32  
 9   Months since last delinquent  9570 non-null   int32  
 10  Number of Open Accounts       9570 non-null   int32  
 11  Number of Credit Problems     9570 non-null   int32  
 12  Current Credit Balance        9570 non-null   int32  
 13  Max

In [15]:
data.head()

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,...,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,445412,709,1167493,5214.74,17,35,6,1,228190,416746,...,1,0,0,0,0,0,0,0,0,0
1,262328,1105,1373190,33295.98,21,8,35,0,229976,850784,...,0,0,0,0,0,0,0,0,0,0
2,99999999,741,2231892,29200.53,14,29,18,1,297996,750090,...,0,0,0,0,0,0,0,0,0,0
3,347666,721,806949,8741.90,12,35,9,0,256329,386958,...,0,0,0,0,0,0,0,0,0,0
4,176220,1105,1373190,20639.70,6,35,15,0,253460,427174,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Data Normalization

xMean = np.mean(data, axis=0)
xDev = np.std(data, axis=0)
xNorm = (data - xMean) / xDev

x = data.values #returns numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
xMinMax = pd.DataFrame(x_scaled)
xNoNorm = data

In [17]:
xNorm

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,...,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,-0.356302,-0.286947,-0.284779,-1.106690,-0.091192,-0.014045,-1.027183,1.767085,-0.191812,-0.048872,...,4.150731,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,-0.245978,-0.052194,-0.04341,-0.025047
1,-0.362060,0.004907,-0.020275,1.203402,0.500574,-1.794795,4.770070,-0.348377,-0.186720,0.001071,...,-0.240921,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,-0.245978,-0.052194,-0.04341,-0.025047
2,2.774845,-0.263363,1.083922,0.866492,-0.535016,-0.409767,1.371680,1.767085,0.007187,-0.010516,...,-0.240921,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,-0.245978,-0.052194,-0.04341,-0.025047
3,-0.359376,-0.278103,-0.748399,-0.816530,-0.830899,-0.014045,-0.427467,-0.348377,-0.111595,-0.052300,...,-0.240921,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,-0.245978,-0.052194,-0.04341,-0.025047
4,-0.364768,0.004907,-0.020275,0.162238,-1.718547,-0.014045,0.771964,-0.348377,-0.119774,-0.047672,...,-0.240921,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,-0.245978,-0.052194,-0.04341,-0.025047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.365794,-0.267048,-0.468116,-0.467709,-1.422664,0.315723,0.572059,-0.348377,-0.171446,-0.027304,...,-0.240921,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,4.065399,-0.052194,-0.04341,-0.025047
9996,-0.357725,0.004907,-0.020275,-0.952873,-0.387075,-0.014045,-1.626899,-0.348377,0.105712,-0.048245,...,-0.240921,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,-0.245978,-0.052194,-0.04341,-0.025047
9997,-0.365453,-0.255993,-0.713877,-0.661133,-0.091192,-0.014045,-0.827278,-0.348377,-0.624258,-0.037233,...,-0.240921,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,4.065399,-0.052194,-0.04341,-0.025047
9998,-0.361992,-0.269259,0.049645,0.793514,-1.126782,-0.014045,2.171301,-0.348377,0.083775,0.046492,...,-0.240921,-0.101191,-0.185092,-0.076029,-0.060586,-0.033923,-0.245978,-0.052194,-0.04341,-0.025047


In [18]:
data_versions = []
# Define three cases we will study
for x in [xNorm, xMinMax, xNoNorm]:
    version = {}
    version['x_train'], version['x_test'], version['y_train'], version['y_test'] = train_test_split(x, y, test_size= 0.25, random_state=13)
    data_versions.append(version)

In [19]:
# Initialize the classifiers that we will be testing
classifiers = [SGDClassifier(), LogisticRegression(), SVC()]

for i, data_version in enumerate(data_versions):
    print("Evaluating the models with data from the version #{}".format(i+1))
    for j, clf in enumerate(classifiers):
        
        #train model with train data
        clf.fit(X=data_version['x_train'], y=data_version['y_train'])
        
        #predict test data
        predictions = clf.predict(X=data_version['x_test'])
        
        #calculate the accuracy
        accuracy = accuracy_score(data_version['y_test'], predictions)

        print("\t Classifier #{} achieved {} accuracy on test data.".format(j+1, accuracy))

Evaluating the models with data from the version #1
	 Classifier #1 achieved 0.8123694107814459 accuracy on test data.
	 Classifier #2 achieved 0.8119515252820727 accuracy on test data.
	 Classifier #3 achieved 0.8111157542833264 accuracy on test data.
Evaluating the models with data from the version #2
	 Classifier #1 achieved 0.8140409527789386 accuracy on test data.
	 Classifier #2 achieved 0.8136230672795653 accuracy on test data.
	 Classifier #3 achieved 0.8140409527789386 accuracy on test data.
Evaluating the models with data from the version #3
	 Classifier #1 achieved 0.7317175094024237 accuracy on test data.
	 Classifier #2 achieved 0.8069368992895947 accuracy on test data.
	 Classifier #3 achieved 0.7718345173422482 accuracy on test data.


In [20]:
# Implement Cross Validation using Logistic Regression classifier
# Using xMinMax dataframe

clf = LogisticRegression()

# 5 fold cross validation
scores = cross_val_score(clf, xMinMax, y, verbose=1, cv=5)
print(scores)
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[0.81713689 0.81818182 0.82445141 0.81922675 0.82183908]
Accuracy: 0.820 (+/- 0.005)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s finished
